In [1]:
# !pip install snowflake-connector-python

In [2]:
import snowflake.connector
import pandas as pd
import json

'''user = REFRACT.FOSFOR@LNTINFOTECH.COM
password = Password321
account = fya62509.us-east-1
warehouse = FOSFOR_REFRACT
database = FOSFOR_REFRACT
schema = SALES
role = FOSFOR_REFRACT'''    

# Snowflake connection parameters
snowflake_account = 'rwb76633'
snowflake_user = 'Ravi'
snowflake_password = 'Kayakave@1983'
snowflake_database = 'TUTORIAL_DB'
snowflake_schema = 'DATA_SCHEMA'
snowflake_stage = 'MAGICODER2'
role = 'TEST_ROLE'

# snowflake_account = 'fya62509.us-east-1'
# snowflake_user = 'REFRACT.FOSFOR@LNTINFOTECH.COM'
# snowflake_password = 'Password321'
# snowflake_database = 'FOSFOR_REFRACT'
# snowflake_schema = 'SALES'
# snowflake_stage = 'MAGICODER'
 
# Connect to Snowflake
conn = snowflake.connector.connect(
   user=snowflake_user,
   password=snowflake_password,
   account=snowflake_account,
   session_parameters={
      'PYTHON_CONNECTOR_QUERY_RESULT_FORMAT': 'json'
   },
   database=snowflake_database,
   schema=snowflake_schema,
   role=role,
)




In [3]:
conn

In [3]:
#! ls -lrt /llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66

In [4]:
# Create a cursor object
cur = conn.cursor()
 
# Upload file to stage (example using Pandas)
#file_path = '/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66/special_tokens_map.json'
# df = pd.read_csv(file_path)
#cur.execute(f"PUT file://{file_path} @{snowflake_stage}/magicoder_model AUTO_COMPRESS = FALSE")
#cur.execute("Select * from Sales")

In [180]:
prompt= """You are an exceptional dbt analytics engineer writes dbt models, also creates very precise and accurate functional documentation to ensure any layman understands the code written by you.
        follow instruction and response accuratly. finally only generate respose without instruction. 
        
        @@Instruction
        this is the sample dbt model
        with customer_revenue as (
        select
        customer_id,
        sum(amount) as total_revenue
        from
        {{ ref('orders') }}
        group by 1
        )
        
        select
        c.customer_id,
        c.first_name,
        c.last_name,
        cr.total_revenue
        from
        {{ source('master_data','customers') }} c
        left join customer_revenue cr on c.customer_id = cr.customer_id

        @@Response
        Below is the sample dbt model documentation generated for above instruction dbt model:
        
        * Overview: This dbt model calculates the total revenue per customer by leveraging the data from the orders model and customers source. 
        * Dependencies:
        * orders: dbt model that contains information about individual orders, including the customer_id and amount.
        * customers: dbt source contains customer details, including customer_id, first_name, and last_name.
        * Transformation:
        * The CTE 'customer_revenue' aggregates the total revenue for each customer by summing the amount from the orders model. It utilizes the customer_id field for grouping.
        * The final output includes customer details from the customers table (c) and their corresponding total revenue from the customer_revenue CTE (cr). The join is performed using the customer_id field.
        * Example Usage:	The resulting model can be used for various analytical purposes, such as identifying high-value customers or generating reports on customer revenue.

        @@ Instruction
        create similar dbt model documentation for following dbt model code
            "create document for following dbt code {% snapshot users_snapshot %}  

    {{
      config(      
        target_schema='snapshots',      
        strategy='timestamp',      
        unique_key='id',      
        updated_at='updated_at'    
      )  
    }}  

      select * 
      from {{ source('raw','users') }}

    {% endsnapshot %}"""


In [186]:
prompt = r"""Construct a DBT view model in the 'staging' schema, linking 'jaffle_shop' orders with payment information from 'stg_payments', summarizing successful payments per order, and merging with order details to handle potential missing payment scenarios."""

In [187]:
prompt = prompt.replace("'", "''")
prompt = prompt.replace("\n", " ")

In [167]:
prompt

"Generate documentation for following dbt code  {{ config(materialized=''table'') }}  with source as (     select * from {{ source(''src_table'', ''employees'') }} ),  parsed as (     select         id,         PARSE_JSON(json_column):field1::varchar as field1,         PARSE_JSON(json_column):field2::int as field2,         PARSE_JSON(json_column):field3::timestamp as field3     from source )  select * from parsed"

In [101]:
prompt = "Generate bdt code to join supply and demand column"

In [86]:
temp_arg ={"prompt":prompt, "max_tokens": 2000, "temperature": 0, "model": "/data/magicoder_model"}

In [87]:
temp_arg

{'prompt': "Construct a DBT view model in the 'staging' schema, linking 'jaffle_shop' orders with payment information from 'stg_payments', summarizing successful payments per order, and merging with order details to handle potential missing payment scenarios.",
 'max_tokens': 2000,
 'temperature': 0,
 'model': '/data/magicoder_model'}

In [113]:
type(arg)

str

In [188]:
arg = '{"prompt":'+'"'+prompt+'"'+', "max_tokens": 2000, "temperature": 0, "model": "/data/magicoder_model"}'
warehouse = 'USE WAREHOUSE TUTORIAL_WAREHOUSE;'
query = f"SELECT magicoder_udf(parse_json('{arg}'));"
print(query)

SELECT magicoder_udf(parse_json('{"prompt":"Construct a DBT view model in the ''staging'' schema, linking ''jaffle_shop'' orders with payment information from ''stg_payments'', summarizing successful payments per order, and merging with order details to handle potential missing payment scenarios.", "max_tokens": 2000, "temperature": 0, "model": "/data/magicoder_model"}'));


In [189]:
cur.execute(warehouse)
cur.execute(query)
result = cur.fetchall()
print(json.loads(result[0][0])['choices'][0]['text'])




```sql
{{ config(
    materialized='view',
    schema='staging'
) }}

with orders as (
    select * from {{ ref('jaffle_shop_orders') }}
),

payments as (
    select * from {{ ref('stg_payments') }}
),

order_payments as (
    select 
        orders.order_id,
        orders.customer_id,
        orders.order_date,
        orders.status,
        sum(payments.amount) as total_amount_paid
    from orders
    left join payments
    on orders.order_id = payments.order_id
    where payments.status = 'success'
    group by 1, 2, 3, 4
)

select * from order_payments
```

This DBT view model is named 'order_payments_summary' and it is constructed by joining the 'jaffle_shop_orders' and 'stg_payments' tables, filtering for successful payments, and summarizing the total amount paid per order. The result is then merged with the order details to handle potential missing payment scenarios.


In [158]:
result

[('{\n  "choices": [\n    {\n      "finish_reason": "stop",\n      "index": 0,\n      "logprobs": null,\n      "text": ""\n    }\n  ],\n  "created": 900528,\n  "id": "cmpl-691f59d7581040858ef60c101927dfdd",\n  "model": "/data/magicoder_model",\n  "object": "text_completion",\n  "usage": {\n    "completion_tokens": 1,\n    "prompt_tokens": 74,\n    "total_tokens": 75\n  }\n}',)]

In [7]:
import requests
import json

# Obtain a session token.
token_data = conn._rest._token_request('ISSUE')
token_extract = token_data['data']['sessionToken']

# Create a request to the ingress endpoint with authz.
token = f'\"{token_extract}\"'
# Set this to the ingress endpoint URL for your service
url = 'http://gowiocnz-zqiseam-ltifosforscsaws.snowflakecomputing.app/vllm/magicoder/v1/completions'

In [8]:
headers = {
  'Content-Type': 'application/json', 'Authorization': f'Snowflake Token={token}'
}
print(headers)
print(url)
#print(payload)

{'Content-Type': 'application/json', 'Authorization': 'Snowflake Token="ver:1-hint:14544919633592326-ETMsDgAAAY5Vep50ABRBRVMvQ0JDL1BLQ1M1UGFkZGluZwEAABAAEMk0A8Nx2TZnVqLTbZSljBcAAACA48T2Lr8JKf/dNoQ+63nJH9+QxeHcASiPik2NSxujbZIIjM54+IsUlHcpGUPDwSwQ1JRZTrKIgY0Yuui3WXfMCELevmH/ouvrrg2R/v+F9ovWKUZG0EcY9IHo/Wvqgd1AEOopoYMmfVV+nCaf6UMf9zYA8pcF9Bp2QhsVDUBPBOEAFLlv+fQcuY44iMx5FEtWCNf9RUCD"'}
http://gowiocnz-zqiseam-ltifosforscsaws.snowflakecomputing.app/vllm/magicoder/v1/completions


In [9]:
payload = json.dumps({
  "prompt": "write a python code to check whether given string is pallindrome",
  "max_tokens": 2400,
  "temperature": 0,
  "model": "/data/magicoder_model"
})
headers = {
  'Content-Type': 'application/json', 'Authorization': f'Snowflake Token={token}'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)


{"detail":"Method Not Allowed"}
